In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from collections import Counter
import csv
import math

In [2]:
filename = "2019_NE_TFIDF.csv"
k = 4

In [3]:
df = pd.read_csv(filename, encoding="utf-8-sig")
news_topic = df['topics'].tolist()
df

,date,press,category,headline,url,text,img,N_list,topics
0,2019/01/01,한겨레,정치,문희상 “임시정부 100년…새 역사 시작하는 새해 될 것”,http://www.hani.co.kr/arti/politics/politics_g...,"문희상 국회의장이 2019년 새해를 “대한민국 임시정부 100년, 임시의정원 100...",['http://img.hani.co.kr/imgdb/resize/2018/1231...,"[['문희상', '국회', '의장', '새해', '대한민국', '임시', '정부',...","의장, 대한민국, 신년사, 질곡, 임시, 절체절명, 역경, 퇴보, 국회, 혼신"
1,2019/01/01,세계일보,"경제,지역",경남 승강기안전공단 지역과 상생 동반성장 활동,http://www.segye.com/content/html/2018/12/31/2...,경남 한국승강기안전공단이 지역과 상생하며 동반성장하는 다양한 활동에 나서고 있다. ...,[],"[['경남', '한국', '승강기', '안전', '공단', '지역', '상생', '...","지역, 인재, 채용, 공단, 자매결연, 신규, 농산물, 농촌, 진주, 상생"
2,2019/01/01,중앙일보,정치,문 대통령 신년사 “한 분 한 분의 삶 나아지도록 노력하겠다”,http://news.joins.com/article/olink/22844371,"문재인 대통령은 1일 “이 겨울, 더 따뜻하게 세상을 밝히라는 촛불의 마음 결코 잊...",['https://pds.joins.com/news/component/htmlpho...,"[['문재인', '대통령', '겨울', '세상', '촛불', '마음', '삶', '...","신년사, 대통령, 평화, 새해, 오찬, 지도부, 마지막, 여당, 국정, 마음"
3,2019/01/01,국민일보,"사회,정치","“그 비서 몸매 좋던데”…국회 페미 “페미니즘=시대정신, 국회부터 바뀌어야”",http://news.kmib.co.kr/article/view.asp?arcid=...,지난해 8월 16일 국회 내에 신설된 익명 단체인 ‘국회 페미’가 최근 언론 인터뷰...,['http://image.kmib.co.kr/online_image/2019/01...,"[['지난해', '국회', '신설', '익명', '단체', '국회', '페', '미...","여성, 국회, 행비, 남성, 의원, 외모, 이도, 보좌관, 비서, 차별"
4,2019/01/01,중앙일보,정치,"공무원 업무추진비 사용, 휴일·심야 증빙자료 내야",http://news.joins.com/article/olink/22844373,지난해 9월 정치권에서 불거진 청와대 업무추진비 논란으로 홍역을 앓은 정부가 올해부...,['https://pds.joins.com/blank.gif'],"[['지난해', '정치', '청와대', '업무', '추진', '논란', '홍역', ...","사용, 증빙, 업무, 주점, 추진, 심야, 지침, 자료, 제한, 근무지"
...,...,...,...,...,...,...,...,...,...
106856,2019-06-30,세계일보,"정치,국제",[WT논평]The reason why we must denuclearize Nort...,http://www.segye.com/content/html/2019/06/30/2...,By Robert R. Monroe (former director of the De...,[],"[[], ['합의', '시바', '삐', '매듭', '필요', '대다수'], ['상...","핵무기, 이란, 세계, 불량, 나라, 국가, 존재, 북한, 공포, 입수"
106857,2019-06-30,세계일보,"국제,정치","[사설] 日 경제보복 카드 우려, 정부는 전략적 대응 모색하길",http://www.segye.com/content/html/2019/06/30/2...,일본이 강제징용 판결에 대해 경제보복 카드를 꺼내든다고 한다. 일본 정부가 TV·스...,['//img.segye.com/content/image/2019/06/30/201...,"[['일본', '강제', '징용', '판결', '경제', '보복', '카드'], [...","보복, 일본, 양국, 아베, 갈등, 관계, 수출, 조치, 매각, 곤란"
106858,2019-06-30,세계일보,정치,[사설] 84일 만의 국회 본회의… ‘완전한 정상화’ 서두를 때다,http://www.segye.com/content/html/2019/06/30/2...,국회가 지난 28일 자유한국당을 포함한 여야 5당이 모두 참석한 가운데 본회의를 열...,['//img.segye.com/content/image/2019/06/28/201...,"[['국회', '자유한국당', '포함', '여야', '당', '참석', '가운데',...","여야, 본회의, 국회, 한국당, 상임위, 추경, 특위, 의사, 처리, 일정"
106859,2019-06-30,세계일보,정치,"[사설] 남·북·미 정상 판문점 회동, 실질적 비핵화로 이어져야",http://www.segye.com/content/html/2019/06/30/2...,분단·냉전 상징서 역사적 만남 / 트럼프 “북·미 실무협상 할 것” / 北 비핵화 ...,['//img.segye.com/content/image/2019/06/30/201...,"[['분단', '냉전', '상징', '역사', '만남', '트럼프', '북', '미...","협상, 트럼프, 위원장, 대통령, 정상, 실무, 북한, 청산, 회담, 회동"


In [4]:
# CountVectorizer or TfidfVectorizer로 토큰화
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(news_topic)

# l2 정규화
X = normalize(X)

# k-means 알고리즘 적용
print("waiting for KMeans")
kmeans = KMeans(n_clusters=k).fit(X)

# trained labels and cluster centers
labels = kmeans.labels_
centers = kmeans.cluster_centers_

# labels에 merge
df['labels'] = labels

hot_topic_index = Counter(df['labels']).most_common()
hot_topic = dict(hot_topic_index)

index = []
for row in df['labels']:
    index.append(hot_topic[row])

df['cnt'] = index

df.sort_values(by='cnt', ascending=False).to_csv("K_"+str(k)+"_CV_" + filename, encoding="utf-8-sig", mode='w', index=None)
print("Done")

waiting for KMeans
Done
